<a href="https://colab.research.google.com/github/nourceneB/delta/blob/master/tutorial_notebooks/Tut7_RNN_for_Text_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Humboldt-WI/delta/blob/master/tutorial_notebooks/Tut7_RNN_for_Text_Student.ipynb)

# Tutorial 7: RNNs for Text

<br>
<img src="https://i.imgur.com/pPPLHNn.png" width="900">
<br>
<br>

## Prediction of online review sentiment

Sentiment analysis is one of the most popular applications for text classification. It is also interesting from a business perspective. For example, many companies have an interest in analyzing text data emerging in social media to understand how consumers value their products, service, brands, etc.

The goal of sentiment analysis is to model the polarity of a piece of text, whether it is rather positive or rather negative. We can frame this task as a binary classification problem, with labels of one and zero indicating positive or negative sentiment, respectively. That is the approach we will take today. Other options  could involve modeling a three-level target (positive, neutral, negative) or a numeric target variable the values of which represent different strengths of polarity (e.g., between +5 and -5). Whenever approaching the sentiment analysis task by supervised learning, we depend on having some data with sentiment labels. That is often the real challenge in practice — where do the labels come from? — and explains why many labeled data sets re-occur in papers.

We will apply different modeling approaches to predict review sentiment, from a simple dictionary-based approach over conventional supervised machine learning to several deep learning techniques. Here is the outline of the session.
  
 1. Preliminaries
 2. Dictionary-based sentiment analysis
 3. Text data representation
 4. Deep learning based text classifiers<br>
    - Feed-forward neural network <br>
    - Recurrent neural network based on GRU<br>
    - Bidirectional GRU<br>
    - GRU with pre-trained IMDB embeddings<br>

Let's get started.

## 1. Preliminaries

To set up our environment we load the IMDB 50K review data set and also the list of cleaned reviews. We then add the cleaned reviews to the data set to have everything in one place. It is a good idea to examine a few reviews and make sure that the original version and the cleaned version match. Ones this is confirmed, you can safely discard the raw review text to save memory. Finally, we update the coding of our target variable and encode positive and negative reviews as one and zero, respectively.  

The notebook sets a new milestone in terms of demand for computational resources. We recommend running the notebook  in **Colab or another cloud-based platform of your choice**.  

In [4]:
# Import standard libraries
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt

In [5]:
# Create a global variable to idicate whether the notebook is run in Colab
import sys
IN_COLAB = 'google.colab' in sys.modules

# Configure variables pointing to directories and stored files
if IN_COLAB:
    # Mount Google-Drive
    from google.colab import drive
    drive.mount('/content/drive')
    DATA_DIR = '/content/drive/My Drive/ProDok/'  # adjust to Google drive folder with the data if applicable
else:
    DATA_DIR = './' # adjust to the directory where data is stored on your machine (if running the notebook locally)

sys.path.append(DATA_DIR)

CLEAN_REVIEW = DATA_DIR + 'imdb_clean_full_v2.pkl'   # List with tokenized reviews after standard NLP preparation
IMBD_EMBEDDINGS = DATA_DIR + 'w2v_imdb_full_d100_e500.model'

Mounted at /content/drive


### Loading the data
We load the data frame with the original and cleaned reviews. The original version does not matter for this session. We will delete them to save memory.

In [7]:
with open(CLEAN_REVIEW,'rb') as path_name:
    df = pd.read_pickle(path_name)
df.info()

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/ProDok/imdb_clean_full_v2.pkl'

In [ ]:
# Drop original review text
df.drop('review', axis=1, inplace=True)

# Binary-encode the target variable
df['sentiment'] = df['sentiment'].map({'positive' : 1, 'negative': 0})
print(df['sentiment'].value_counts())

### Downsampling the data to increase speed
One more thing before moving on. You should decide whether you want to proceed with the full data frame (i.e., 50K reviews) or draw a random sample to decrease the runtime of the following examples. Using all the data is feasible on any descent computer but prepare for a bit of waiting when training our neural networks. Here is a little bit of code to reduce the amount of data.

In [ ]:
# Draw a random sample of n reviews to increase the speed of the following steps
n = 5000
np.random.seed(111)
ix = np.random.randint(0, high=df.shape[0]-1, size=n)
df = df.loc[ix, :]
df.reset_index(drop=True, inplace=True)

In [ ]:
print(df)

Before moving on with developing sentiment classification models, we partition our data into a training and a hold-out test set.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['review_clean'], df['sentiment'], test_size=0.25, random_state=111)

## 2. Dictionary-based sentiment analysis

A simple approach to rate the sentiment of a text is to literally model it as the sum of its parts through the sentiment of each word. `pysentiment2` is a package containing several English word sentiment dictionaries. Word scores are either negative (-1), neutral (0) or positive (1). On a sentence level the polarity score is the average of positive and negative words (neutral words are ignored). The Harvard IV-4 dictionary, which we will use, consists of 3,642 coded words. Note that you will need to install the library before being able to run the following code.

In [ ]:
import pysentiment2 as ps
dc = ps.HIV4() # import Harvard IV-4 dictionary

def get_sentiment_score(text):
    score = round(dc.get_score(dc.tokenize(text))['Polarity'], 2)
    return score

We look up the sentiment score for each word in turn and sum up the sentiment values over words. Here are a few examples. Quite easy, isn't it?

In [ ]:
#* Some examples how to rate texts. Larger values indicate stronger positive feelings
examples = ["What a marvelous evening, the weather is simply delightful. Wonderful!",
            "I am devastated, the donuts are not what they once were – disgusting.",
            "To be or not to be?"
           ]
for txt in examples:
    print(txt, "\t -> Sentiment score:", get_sentiment_score(txt))

In [ ]:
get_sentiment_score('bad')

### Sentiment score calculation  
Since we already have a data frame, why not add the sentiment score of every review as a new column. This is a nice use case for the `.apply()` function that Pandas data frames support. We score the cleaned version of the review. If you fancy a little exercise, consider to also score the original review text and compare the differences between the two scores. You could then identify reviews where the sentiment scores differ substantially between the original and cleaned text. That might point to some issues in our data cleaning.

In [ ]:
# Add the Afinn scores to our data frame
# Caution: if you use the full data set of 50K reviews, the scoring will take a while.
start = time.time()
df['sentiment_score'] = df['review_clean'].apply(get_sentiment_score)
end = time.time()

print('Processed {} reviews in {:.0f} sec.'.format(df.shape[0], end-start))

In [ ]:
df['sentiment_score'].hist();

In [ ]:
print(df['sentiment_score'].describe()) # overall rather positive

### Sentiment classifier assessment
We can treat the sentiment scores as class predictions. Applying a classification cut-off of zero, we posit that every review with a positive score is classified as positive, and negative otherwise. We can then examine the predictive accuracy of the dictionary-based classifier using standard performance measures for binary classification. Since we will do so a couple of times, we put the code in a function.

In [ ]:
# Assess sentiment classification models
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

def assess_sentiment_classifier(ytest, yhat, cut_off=0.5, plot_roc=True):
    """
        Helper function to assess a classification model in terms of the AUC
        and classification accuracy. We calculate the latter by comapring
        continuous (e.g., probabilistic) classifier predictions to a cut_off.
        Cases where the prediction exceeds the cut-off are classified as
        positive.
    """
    # Calculate discrete class predictions
    yhat_c = np.where(yhat>cut_off, 1, 0)

    # Calculate classification accuracy and AUC
    acc = accuracy_score(ytest, yhat_c)
    auc = roc_auc_score(ytest, yhat)

    # Confusion matrix
    cmat = confusion_matrix(ytest, yhat_c)

    # ROC analysis
    if plot_roc==True:
        fpr, tpr, _ = roc_curve(ytest, yhat)
        plt.plot(fpr,tpr, label="AUC={:.4}".format(auc));
        plt.plot([0, 1], [0, 1], "r--")
        plt.ylabel('True positive rate')
        plt.xlabel('False positive rate')
        plt.legend(loc='lower right')
        plt.show();

    return(auc, acc, cmat)

In [ ]:
# Assess lexicon-based sentiment classifier
auc, acc, cmat = assess_sentiment_classifier(ytest=df['sentiment'], yhat=df['sentiment_score'], cut_off=0)
print("Lexicon-based sentiment classifier:\tAUC={:.4f}\tAccuracy={:.4f}".format(auc, acc))

In [ ]:
# Confusion matrix
cmat

The ROC curve looks good, whereas the confusion matrix suggests that our dictionary-based classifier is biased towards positive reviews. Note that your result might differ depending on which data you are using (all reviews, random sample). Finally, note that our performance estimates come from the whole data. That is ok since the lexicon-based approach does not involve training. Therefore, there is no risk of overfitting and no need for data partitioning.
<br>Since we are about to create more models, it makes sense to create a container in which we store the results of different models. Good options are either a dictionary or a data set. In this notebook, we use the latter approach. To ensure comparability across different approaches, we will re-calculate the performance of the lexicon-based model for the observations of the test set.

In [ ]:
# Extract test set cases
yhat = df.loc[X_test.index, 'sentiment_score']

# Compute test set performance
auc, acc, _ = assess_sentiment_classifier(y_test, yhat, cut_off=0, plot_roc=False)

# We will create many more models. Let's put every model's score into a data frame
df_scores = pd.DataFrame(index=['Acc', 'AUC'], columns=['Lexicon'], data=[acc, auc])
print(df_scores)

## 3. Text data representation
For the next models, we need to represent the text data in a numeric format, which the learning algorithms can process. The specific form of the data depends on the type of learning algorithm. Traditional machine learning algorithms use some form of count vectorization, meaning that the words in a text are replaced by their number of occurrence in that text. This format is also called the **document-term matrix (DTM)**. A tabular data structure in which rows are documents (e.g., reviews in our case) and the columns are the unique words in the vocabulary. Given that a document is seen as a **bag of words**, BoW is just another popular name to refer to corresponding text representations. You can find many sources on the Internet such as [this one](https://www.analyticsvidhya.com/blog/2020/02/quick-introduction-bag-of-words-bow-tf-idf/) to obtain a more comprehensive explanation of the BoW model if interested. Since our focus is deep learning, we will not go there.  However, let's note one important caveat of the BoW approach. The size of the vocabulary is typically very large. For example, according this [Wikipedia page](https://en.wikipedia.org/wiki/List_of_dictionaries_by_number_of_words), there are about 470K unique words in the English language. Making every unique word a feature in the DTM, our matrix will be high dimensional. Stemming can help a bit but **high-dimensionality** is one key disadvantage of representing data in form of a DTM. We will address this issue by developing a LASSO classifier that is robust toward dimensionality.
<br>
One advantage of word vectors over the BoW approach is that they avoid high dimensionality by embedding words in a lower dimensional space. Using word vectors, a document is represented as a **sequence of words** in which each word is represented by its embedding. This is the data structure that we use for our deep learning-based models.
<br>
In order to create the two text representations for traditional machine learning classifiers and deep learning classifiers, we use the class `Tokenizer` from the `Keras` text processing API. Libraries like `scikit-learn` and others provide similar functionality (see, e.g., [here](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)). When using `Keras`, our first task is to create a tokenizer object and 'train' it, a bit like with models, on our text data. To that end, we use the function `fit_on_texts`. Afterwards, we can use two functions for converting our text into a BoW or a sequence of words for machine and deep learning models, respectively. These functions go by the telling names `texts_to_matrix()` and `texts_to_sequences()`, respectively. We illustrate both of them in the following. But let's first build the vocabulary. We will need to decide how large our vocabulary should be. In practice, that can be a tough decision, the pros and cons being potentially higher accuracy when using many/all words in the text corpus versus faster computations when dropping some infrequent words. We just make a somewhat arbitrary choice in this notebook but remember that the **vocabulary size** is a meta-parameter that you will want to examine when working on a serious case.

In [ ]:
# Build vocabulary using Keras
from tensorflow.keras.preprocessing.text import Tokenizer
NUM_WORDS = 2500  # 2.5K is a rather small vocabulary. This choice is only to increase speed. Increase the setting and check whether you can raise performance

# Create tokenizer object and build vocab from the training set
tok = Tokenizer(NUM_WORDS, oov_token=1)  # We fit the tokenizer to the training set reviews. The test set might include
tok.fit_on_texts(X_train)  # words that are not part of the training data. The argument oov_token ensures that such new words are mapped to the specified index

The fitted tokenizer object exposes several methods that summarize what was learned and facilitates looking at the internal data from different perspectives. Here are a few examples:

In [ ]:
# On how many documents did we train?
print(tok.document_count)

In [ ]:
# How many unique words?
len(tok.word_counts)

In [ ]:
# A dictionary of words storing in how many documents a word appeared
word = 'film'
n = tok.word_docs[word]
print('The word <{}> appeared in {} reviews.'.format(word, n))

We embed words in a lower dimensional vector space and represent text as a sequence of embedding vectors. More specifically, the input to a deep network is a sequence of one-hot encoded words. The mapping of words to embedding vectors happens within the first layer of the network, the **embedding layer**. Few examples below. For now, the fact that the mapping is part of the network architecture means that our pre-processing consists of mapping the sequence of words in a review to one-hot-vectors. More precisely, we map the words to a **sequence of integer values**. That is more efficient. Knowing the size of our vocabulary, V, a one-hot-vector is a vector of V-1 zeros and a single value if one. We can encode that by an integer that tells us the index of the one in the one-hot vector. `Keras` provides this type of text preparation via the function `texts_to_sequences()`.  

In [ ]:
# Convert training set reviews to sequences of integer values
X_tr_int = tok.texts_to_sequences(X_train)
print('Type of result is: ', type(X_tr_int))
print('Length of our list: ', len(X_tr_int))

In [ ]:
X_tr_int[0][:9]

We can convert-back the integers to words using the methods of the fitted tokenizer as follows:

In [ ]:
demo = [tok.index_word[token] for token in X_tr_int[0][:9]]
demo

And again back to integers...

In [ ]:
[tok.word_index[token] for token in demo]

Two remarks are appropriate at this point. First,  `text_to_sequences()` also randomizes the data. Therefore, when looking at the first training example, it is again a different example (review) compared to the one that was in the first position after calling `text_to_matrix()`. So don't be confused to see another text. Second, the mapping from words to integers in `Keras` uses a *hashing function* and this function does not guarantee that words get mapped to unique integers. Collisions are possible. You can check the `Keras` documentations for details; simply search for *hashing trick*. It is not dramatically important at this point but when you play with codes like the above, mapping words to integers and than back to words, and back to integer, you might realize some odd behavior of different words getting mapped to the same integer. Show you ever observe such behavior, now you know it's coming from the hashing trick.

The data is almost ready. However, the `Keras` layers that we will use later expect the input data to have a fixed, pre-defined shape. Our reviews differ substantially in length. So, the next task on our todo list is to pad the reviews and ensure a consistent sequence length. We could consider artificially making every review as long as the longest one. This way, we would not lose any data.  

In [ ]:
# Determine the maximum review length in the training set
max_review_length = max([len(review) for review in X_tr_int])
print('The longest review of the training set has {} words.'.format(max_review_length))

Standard practice in NLP is to embed words in a vector space. Considering an embedding dimension of, e.g., 100, each word in the input data (i.e., review) will be mapped to a 100 dim vector. Working with a large embedding dimension and long sequences will result in slow training. Since we care more about illustrating concepts than building the best possible sentiment classifier, we will set an upper bound on the text length and pad reviews accordingly. All reviews that are shorter than our upper bound will be padded with zeros. Longer reviews will be truncated. In practice, you would need to experiment carefully whether and how much truncating the data hurts performance.

In [ ]:
# Upper bound of the review length for padding
from keras.preprocessing.sequence import pad_sequences
MAX_REVIEW_LENGTH = 400

X_tr_int_pad = pad_sequences(X_tr_int, MAX_REVIEW_LENGTH)

In [ ]:
X_tr_int_pad.shape

So far, we dealt only with the training data. So it is about time to also process the test data.

In [ ]:
# Encode and pad the test data
X_ts_int = tok.texts_to_sequences(X_test)  # Due to oov_token argument, new words will be mapped to 1
X_ts_int_pad = pad_sequences(X_ts_int, MAX_REVIEW_LENGTH)

In [ ]:
# Structure of the prepared training and test data
X_tr_int_pad.shape, y_train.shape, X_ts_int_pad.shape, y_test.shape

## 4. Deep learning-based text classifiers
If we can use LASSO, we can also use a neural network for sentiment prediction. Session P.II.1 provided us with some flavor developing deep networks in `Keras`. The following part will provide many more examples. They will also introduce us to using recurrent and bidirectional architectures. Luckily, `Keras` makes using these fairly sophisticated neural networks relatively easy. Since our data is already of the right form, that is a sequence of integer, we can start right away and without further preparation with building networks.  

#### Embedding layers
When it comes to textual data, a crucial part of the architecture of a neural network is the **embedding layer**. `Keras` embedding layers support two use cases:
* Learning an embedding as part of network training
* Using pre-trained embeddings

We will illustrate both use cases in the reminder.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, Flatten, Dropout, LSTM, Bidirectional
from keras.initializers import Constant

An Embedding is the entry point (i.e., first hidden layer) to a NN for text processing. When creating an embedding layer, you have to specify three arguments:

* input_dim: This is the size of our vocabulary
* output_dim: This is the embedding dimension, or, put differently the size of our word vectors
* input_length: This is the length of our input sequences, i.e., the length of the reviews in words

Knowing that these inputs are crucial further clarifies some previous steps, for example, why we had using padding to ensure a consistent length across all reviews.

In [ ]:
# Some variables to centralize the configuration of deep learning models
NB_HIDDEN = 16        # Hidden nodes / state in fedforward / recurrent NNs
EPOCH = 5             # Passes through the entire data set
BATCH_SIZE = 64 #128  # Batch size
EMBEDDING_DIM = 50    # Embedding dimension
VAL_SPLIT = 0.25      # fraction of the training set used for validation

With these settings, we can create our embedding layer as follows:

In [ ]:
# Create an embedding layer
emb_layer = Embedding(input_dim=NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_REVIEW_LENGTH)

#### Model 1: Feed-forward neural network

Once we have an embedding layer, we can move and stack other layers on top of it. Let's first illustrate that approach with a simple feed-forward network. Importantly, the output of the embedding layer will be a matrix: one vector of size equal to the embedding dimension for each word in the input sequence. If we add a fully-connected (**dense**) layer on top of that, we need to **flatten** the output. If the length of a review is 6 and the embedding dimension is 8, then flattening means that we create a 6x8=42 dimension vector, which represents the input to the dense layer.

In [ ]:
# Feed-forward neural network
fnn = Sequential()
fnn.add(emb_layer)                           # embedding layer
fnn.add(Flatten())                           # flattening layer (to ensure compliance with the input of the next layer)
fnn.add(Dense(NB_HIDDEN, activation='relu')) # fully-connected layer
fnn.add(Dropout(0.2))                        # dropout layer to prevent overfitting
fnn.add(Dense(1, activation='sigmoid'))      # output layer (one unit with sigmoid activation of binary classification)

# compile the model
fnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(fnn.summary())

In [ ]:
# Just for fun, let's recalculate the number of parameters
print('Embedding layer parameters: {}'.format(NUM_WORDS * EMBEDDING_DIM))
print('Flatten layer output shape: {}'.format(MAX_REVIEW_LENGTH * EMBEDDING_DIM))
print('Dense layer parameters: {}'.format(NB_HIDDEN * (MAX_REVIEW_LENGTH * EMBEDDING_DIM)+ NB_HIDDEN))

In [ ]:
# fitting the model
story = fnn.fit(X_tr_int_pad, y_train, batch_size=BATCH_SIZE, epochs=EPOCH, validation_split=VAL_SPLIT)

##### A little bit of infrastructure
The FNN was the first model in a chain of models of increasing sophistication. Sounds promising doesn't it.
Since we are about to train more and more networks, we should develop a little bit of infrastructure code to work with them. Specifically, for each network, we need to produce test set predictions. Also, we would like to examine the development of the loss during training; e.g., to judge whether increasing the number of epochs would make sense. Last, it would be useful to save trained models to disk. After all, we spent quite some time on training them to making a backup in case something goes wrong with out notebook makes a lot of sense. Let's develop some helpers for these tasks.

In [ ]:
def diag_nn(model, story, x_ts, y_ts, plot_roc=True, plot_loss=True):
    '''
        Diagnose fitted keras models by plotting results from the
        story (e.g., development of training loss) and calculating
        classification performance (accuracy & AUC) on the test set
    '''

    # Calculate test set predictions
    yhat = model.predict(X_ts_int_pad)

    # Calling our helper for classifier evaluation
    auc, acc, cmat = assess_sentiment_classifier(y_ts, yhat, cut_off=0.5, plot_roc=plot_roc)
    print("NN test set performance:\tAUC={:.4f}\tAccuracy={:.4f}".format(auc, acc))
    print('Confusion matrix:')
    print(cmat)

    if plot_loss:
        plt.plot(story.history['loss'])
        plt.plot(story.history['val_loss'])
        plt.title('loss evolution')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'validation'], loc='upper left')
        plt.show()

    return (auc, acc, cmat)

In [ ]:
# Assess the performance of the FFN
auc, acc, _ = diag_nn(fnn, story, X_ts_int_pad, y_test, plot_roc=True, plot_loss=True)

# Add results to our data frame to keep track of results
df_scores['FFN'] = [acc, auc]

#### Model 2: Recurrent neural network based on GRU
If you felt that the lecture part on advanced RNNs with gated cells was a little complicated you will be pleased to see that using corresponding models is actually quite easy. <br>
**Disclaimer** We do not promise good results to be easily obtained, but getting a GRU to work is not difficult. In fact, it is easy because we already have almost all we need in place. The following codes is almost identical with that of developing the FNN.
<br>
The only difference is that we use different layers to design a different type of NN, namely a GRU layer. We chose GRU over LSTM because training the former is faster. You can easily adjust the codes to build a LSTM. Simply exchange the type of RNN layer to LSTM. Done.

In [ ]:
# GRU text classifier
gru=sequential()
gru.add(emb_layer)
gru.add(GRU(NB_HIDDEN))
gru.add(Dropout(0.2))
gru.add(Dense(1,activation='sigmoid'))


In [ ]:
gru.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
print(gru.summary())

In [ ]:
# Assessing the first GRU
auc, acc, _ = diag_nn(gru, story, X_ts_int_pad, y_test, plot_roc=True, plot_loss=True)

# Add results to our data frame to keep track of results
df_scores['GRU'] = [acc, auc]

#### Model 3: Bidirectional Recurrent neural network based on GRU

Bidirectional Recurrent Neural Networks (BiRNNs) are an extension of standard RNNs that process input sequences in both forward and backward directions. This dual context allows the network to capture dependencies from both past and future tokens in a sentence, which is particularly beneficial in sentiment analysis where meaning often depends on the full context.

By incorporating both directions, BiRNNs improve contextual understanding and often lead to better performance on natural language tasks.

<br>
<img src="https://i.imgur.com/iweqX2v.png" width="900">
<br>
<br>


In [ ]:
# Bidirectional GRU text classifier
...

In [ ]:
# Assessing the Bidirectional GRU
auc, acc, _ = diag_nn(bi_gru, story, X_ts_int_pad, y_test, plot_roc=True, plot_loss=True)

# Add results to our data frame to keep track of results
df_scores['Bi-GRU'] = [acc, auc]

#### Model 4: LSTM with pre-trained IMDB embeddings

The previous models incorporated an embedding layer. However, training the weights of these layers, that is learning the word embeddings, was part of training the network. You can imagine that embeddings that you learn on-the-fly while training are different from embeddings that result from a model that is specifically designed to learn embeddings such as Word-to-Vec. Weights in Model 1 and Model 2  including the weights of the embedding matrix were trained to predict review sentiment. Word-to-Vec, on the other hand, solves a different prediction task related to the co-occurrences of words in a pre-defined context window. We have trained corresponding weights in the previous coding session P.II.2. It is about time to put these embeddings into action. Our next model will be similar to the previous one but use our own pre-trained IMDB embeddings resulting from applying the `Gensim` implementation of word-to-vec to our data.  

##### Loading pre-trained embeddings

Let's start with loading the `Gensim` embeddings. A version is available in our shared folder.

In [ ]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format(IMBD_EMBEDDINGS, binary=False)

In [ ]:
# Load pretrained W2V embeddings obtained from the IMDB review data set
from gensim.models.keyedvectors import Word2VecKeyedVectors

imdb_index = KeyedVectors.load_word2vec_format(IMBD_EMBEDDINGS, binary=False)

print('Loaded pre-trained embeddings for {} words in {:.0f} sec.'.format(len(imdb_index.key_to_index), time.time()-start))

Pre-trained embeddings are essentially just a bunch of numbers. Every word has its own bunch of numbers or, more formally, numeric representation. Needless to say, the numbers are supposed to carry meaning, capturing syntactic and semantic relationships between words, etc. The embeddings that we just loaded come in the form of a dictionary. In the dictionary, words serve as the key and the corresponding value is the pre-trained embedding of that word. Let's illustrate this using the word *movie* as an example.  

In [ ]:
imdb_index['movie']

Having loaded this type of dictionary, our next task is to look-up the embeddings for each of the words in our vocabulary. In simpler terms, for each word that we encounter in our text, we need to look-up its embedding. We will face the same task when working with Glove embeddings later. So let's wrap-up the code in a helper function, which we can reuse later.

In [ ]:
def get_embedding_matrix(tokenizer, pretrain, vocab_size, verbose=0):
    '''
        Helper function to construct an embedding matrix for
        the focal corpus based on some pre-trained embeddings.
    '''

    dim = 0
    # We will use the function with different types of embeddings. Therefore,
    # we need a condition to determine what is the right way of determining
    # the embedding dimension.
    if isinstance(pretrain, KeyedVectors) or isinstance(pretrain, Word2VecKeyedVectors):
        dim = pretrain.vector_size
    elif isinstance(pretrain, dict):
        dim = next(iter(pretrain.values())).shape[0]  # get embedding of an arbitrary word
    else:
        raise Exception('{} is not supported'.format(type(pretrain)))


    # Initialize embedding matrix
    emb_mat = np.zeros((vocab_size, dim))

    # There will be some words in our corpus for which we lack a pre-trained embedding.
    # In this tutorial, we will simply use a vector of zeros for such words. We also keep
    # track of the words to do some debugging if needed
    oov_words = []
    # Below we use the tokenizer object that created our task vocabulary. This is crucial to ensure
    # that the position of a words in our embedding matrix corresponds to its index in our integer
    # encoded input data
    v = len(tokenizer.word_index)
    start = time.time()
    print('Start embedding process for {} words.'.format(v), flush=True)

    for word, i in tokenizer.word_index.items():
        # try-catch together with a zero-initilaized embedding matrix achieves our rough fix for oov words
        try:
            emb_mat[i] = pretrain[word]
        except:
            oov_words.append(word)
        # Some output that the method is still alive
        if i % 5000 == 0 and verbose>0:
            print('{}/{} words in {} sec'.format(i, v, (time.time()-start)), flush=True)


    print('Created embedding matrix of shape {} in {} min '.format(emb_mat.shape, (time.time()-start)/60))

    print('Encountered {} out-of-vocabulary words.'.format(len(oov_words)))
    return (emb_mat, oov_words)

A few more words on embeddings...

At present, we use embeddings that were obtained from the same corpus, namely the IMDB movie review data set, as the one we are working with right now. That is not common. Typically, the pre-training was done on some other — much larger — corpora. Remember that the very purpose of using pre-trained embeddings is that we hope the pre-trained embeddings to embody some information about word relationships that also prove valuable for our task. The larger the pre-trainind corpus the better. That said, there are some pitfalls that we need to be aware of. For example, when working with two different corpora, that used for pre-training embeddings and that used in the target task, the pre-training corpus will include words that do not appear in our corpus. That is less of a problem. More important, however, our corpus might — and often will — include some words for which we lack an embedding. Addressing this issue in a satisfactory manner is out of the scope of this tutorial. Pre-training an embedding for unknown words is a way forward but not a silver bullet. We will simply apply a rough fix and map out-of-vocabulary words to an embedding vector of zeros. Last, and most importantly, a matrix of pre-trained embeddings functions like a lookup table. The Keras embedding layer will not compute a dot product between a one-hot encoded input word and the embedding matrix because this would be inefficient. Instead, Keras expects to find the embedding of a word with index i in the i'th row of the embedding matrix. Therefore, **it is critical that index of a word in our embedding matrix is the same as in our dictionary of word indices**. Remember that we represent words as integers and sentences as a sequence of integers. Assuming a fictious encoded sentence [2, 44, 21], it would be important that the embedding matrix stores the word vectors of the words, which were mapped to the numbers 2, 44, and 21, respectively, in row 2, 44, and 21, respectively. Make sure you convince yourself that our above helper function achieves this consistency.

Having understood all these details, we can eventually create our embedding matrix using our helper function.

In [ ]:
# Create embedding weight matrix
imdb_embeddings, _ = get_embedding_matrix(tok, imdb_index, NUM_WORDS)

For illustration, let's have a look at the embedding of the word 'movie'.

In [ ]:
imdb_embeddings[tok.word_index['movie'],:]

It's time to build another model, this time using our IMDB embeddings. Specifically, we illustrate how to **initialize the weights** in the `Keras` **embedding layer** with our **pre-trained embeddings**. This is the way in which you use a pre-trained embedding in `Keras`. No matter what type of pre-training embedding you obtained and from where, to use it, you set the embedding layer weights to your embedding matrix. This is why it is so critical to **pay attention to word indices**; just in case we have not mentiond it ;)

In [ ]:
# Create an embedding layer using pre-trained embeddings
...

In [ ]:
# Assessing the first GRU
auc, acc, _ = diag_nn(gru_imdb, story, X_ts_int_pad, y_test, plot_roc=True, plot_loss=True)

# Add results to our data frame to keep track of results
df_scores['GRU_IMDB'] = [acc, auc]

## Conclusions
The examples have covered important concepts in deep learning for NLP and text classification. We developed several deep learning-based text classifiers using Keras and advanced our understanding of word embeddings. We also saw examples of how to use pre-trained word embeddings in downstream tasks, such as sentiment analysis. Advancing the code to approach other text classification systems is straightforward. On the other hand, the results of the Keras models are not yet impressive. Here is th overview of the performance over all classifiers developed in the session.

In [ ]:
print(df_scores)

Considering the performance of the linear model, which is very easy to develop, fast, and memory efficient, we have not yet seen evidence that we need deep learning for the focal sentiment analysis task. It would be a very good exercise and try tuning the more advanced models, starting with proper training for more epochs, to raise their performance and achieve at least competitiveness with the linear model, and maybe the FNN. Good luck ;)